# Import Libraries and Tushare API

In [1]:
import tushare as ts
import pandas as pd
import numpy as np
import datetime as datetime

In [2]:
ts.set_token('e72d522c76cfb8c6fd4ddcb0e2f14027313ae1d6593b3f108332f0ab')
pro = ts.pro_api()
pro = ts.pro_api('e72d522c76cfb8c6fd4ddcb0e2f14027313ae1d6593b3f108332f0ab')

# Dataframe of Chinese Market

1. This dataframe include the basic info, basic financial metrics(P/E, P/B, P/S) of the stocks in Chinese Market. 
    
    
2. I still want to add quick-ratio, EPS, Debt-to-Equity, ROE, EBIT from [fina_indicator] function. However, we need a token interface which has above 5000 points to get the info of whole market.


3. The most recent data we can get is '20200619'.


4. There are 3860 stocks in Tushare API, but some of them miss financial data, so I simply drop rows with nan. The dataframe include 3392 stocks info.


In [4]:
date='20200819'
pro = ts.pro_api()
basic = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,industry, market, exchange')
mv = pro.daily_basic(ts_code='', trade_date=date, fields='ts_code,total_mv,pe,pb, ps')
#df.index=df.ts_code
df1=pd.merge(left=basic, right=mv, how='outer', left_on='ts_code', right_on='ts_code' )
vol = pro.daily(trade_date=date, fields='ts_code, vol')
df2=pd.merge(left=df1, right=vol, how='outer', left_on='ts_code', right_on='ts_code' )
df2=df2.dropna()
#ratios=pro.fina_indicator(ts_code='', trade_date='20180726', fields='ts_code,quick_ratio, eps, ebitda, roe, debt_to_eqt， free_cashflow')
#df3=pd.merge(left=df2, right=ratios, how='outer', left_on='ts_code', right_on='ts_code' )
df2

,ts_code,symbol,name,industry,market,exchange,pe,pb,ps,total_mv,vol
0,000001.SZ,000001,平安银行,银行,主板,SZSE,10.3930,1.0534,2.1240,2.930294e+07,1420928.11
1,000002.SZ,000002,万科A,全国地产,主板,SZSE,8.2010,1.7324,0.8665,3.187906e+07,730576.97
2,000004.SZ,000004,国农科技,互联网,主板,SZSE,1458.0847,3.2372,41.7485,4.519141e+05,19491.72
3,000005.SZ,000005,世纪星源,环境保护,主板,SZSE,18.7241,2.1320,5.9190,3.270879e+05,241494.37
4,000006.SZ,000006,深振业A,区域地产,主板,SZSE,13.8174,1.6717,2.9668,1.106996e+06,474917.91
...,...,...,...,...,...,...,...,...,...,...,...
3942,688589.SH,688589,力合微,半导体,科创板,SSE,129.9932,8.0251,20.4111,5.649000e+05,32398.90
3943,688598.SH,688598,金博股份,矿物制品,科创板,SSE,116.3347,7.4753,37.7250,9.036000e+05,17934.01
3944,688599.SH,688599,天合光能,电气设备,科创板,SSE,52.6211,2.3645,1.4454,3.370883e+06,116225.02
3945,688600.SH,688600,皖仪科技,电器仪表,科创板,SSE,53.7331,4.6957,8.7365,3.573512e+05,11356.74


# Industries of Chinese Market

In [5]:
industry=df2.industry.unique()
print(industry)
print ('Nums of Industries in Chinese Market: ', len(industry))

['银行' '全国地产' '生物制药' '环境保护' '区域地产' '酒店餐饮' '运输设备' '综合类' '建筑工程' '玻璃' '家用电器'
 '其他商业' '元器件' 'IT设备' '其他建材' '汽车服务' '火力发电' '医药商业' '汽车配件' '广告包装' '轻工机械'
 '多元金融' '饲料' '电气设备' '房产服务' '石油加工' '铅锌' '农业综合' '批发业' '通信设备' '旅游景点' '港口'
 '机场' '石油贸易' '空运' '商贸代理' '化学制药' '新型电力' '影视音像' '工程机械' '软件服务' '证券' '化纤' '水泥'
 '专用机械' '供气供热' '农药化肥' '百货' '路桥' '造纸' '医疗保健' '食品' '化工原料' '矿物制品' '水运' '日用化工'
 '文教休闲' '机械基件' '中成药' '汽车整车' '煤炭开采' '铁路' '染料涂料' '白酒' '林业' '水务' '水力发电' '纺织'
 '铝' '旅游服务' '保险' '园区开发' '小金属' '铜' '普钢' '互联网' '特种钢' '种植业' '出版业' '焦炭加工' '啤酒'
 '航空' '公路' '超市连锁' '钢加工' '渔业' '软饮料' '化工机械' '塑料' '红黄酒' '橡胶' '家居用品' '摩托车'
 '黄金' '电器仪表' '服饰' '仓储物流' '纺织机械' '电器连锁' '装修装饰' '半导体' '电信运营' '石油开采' '机床制造'
 '乳制品' '商品城' '农用机械' '公共交通' '陶瓷' '船舶']
Nums of Industries in Chinese Market:  110


# Rank by Market Cap and show Financial metrics of Industry Leader

In [13]:
idx=df2.groupby(['industry'])['total_mv'].transform(max)==df2['total_mv']
df5=df2[idx]
df5=df5.sort_values(by='industry', ascending=False)
pd.set_option('display.max_rows', 110)
df5

,ts_code,symbol,name,industry,market,exchange,pe,pb,ps,total_mv,vol
2669,600547.SH,600547,山东黄金,黄金,主板,SSE,84.8298,4.6058,1.7465,1.093853e+07,278088.88
358,000876.SZ,000876,新希望,饲料,主板,SZSE,24.3328,4.3993,1.4952,1.226860e+07,383718.35
3392,603288.SH,603288,海天味业,食品,主板,SSE,72.2703,25.3222,19.5423,3.868765e+07,72058.18
1694,300285.SZ,300285,国瓷材料,陶瓷,创业板,SZSE,58.8901,7.9466,13.6912,2.947824e+06,269785.15
3128,601398.SH,601398,工商银行,银行,主板,SSE,5.9358,0.7246,2.1672,1.853313e+08,1986050.88
3145,601600.SH,601600,中国铝业,铝,主板,SSE,56.4089,0.9775,0.2526,4.800394e+06,493980.25
2532,600362.SH,600362,江西铜业,铜,主板,SSE,18.7428,0.9481,0.1923,4.622744e+06,132349.73
237,000688.SZ,000688,国城矿业,铅锌,主板,SZSE,125.2060,9.8969,20.9622,2.139260e+06,54904.16
3185,601816.SH,601816,京沪高铁,铁路,主板,SSE,26.1632,1.7037,9.4808,3.123172e+07,870610.81
295,000778.SZ,000778,新兴铸管,钢加工,主板,SZSE,9.2502,0.6420,0.3387,1.384835e+06,334134.81


# Rank by Trading Volume and show Financial metrics of Industry Leader

In [14]:
idx=df2.groupby(['industry'])['vol'].transform(max)==df2['vol']
df4=df2[idx]
df4=df4.sort_values(by='industry', ascending=False)
pd.set_option('display.max_rows', 110)
df4

,ts_code,symbol,name,industry,market,exchange,pe,pb,ps,total_mv,vol
3206,601899.SH,601899,紫金矿业,黄金,主板,SSE,24.6430,2.3642,0.7757,1.055694e+07,2033062.69
836,002385.SZ,002385,大北农,饲料,中小板,SZSE,73.8874,3.9533,2.2877,3.792578e+06,2934973.52
931,002481.SZ,002481,双塔食品,食品,中小板,SZSE,115.9338,7.6953,10.2201,2.168472e+06,445010.66
1694,300285.SZ,300285,国瓷材料,陶瓷,创业板,SZSE,58.8901,7.9466,13.6912,2.947824e+06,269785.15
3109,601288.SH,601288,农业银行,银行,主板,SSE,5.5278,0.6438,1.8691,1.172443e+08,2011989.10
2412,600219.SH,600219,南山铝业,铝,主板,SSE,15.3918,0.6346,1.1612,2.497651e+06,585014.94
199,000630.SZ,000630,铜陵有色,铜,主板,SZSE,24.0223,1.1284,0.2208,2.052674e+06,513425.26
91,000426.SZ,000426,兴业矿业,铅锌,主板,SZSE,128.8666,2.5076,15.0775,1.317267e+06,461006.62
3185,601816.SH,601816,京沪高铁,铁路,主板,SSE,26.1632,1.7037,9.4808,3.123172e+07,870610.81
295,000778.SZ,000778,新兴铸管,钢加工,主板,SZSE,9.2502,0.6420,0.3387,1.384835e+06,334134.81
